<h1> <font color=#00539CFF> Header: Loading tha XML data and importing them in pandas data frame </font></h1>


In [2]:
from tqdm import tqdm
import math
import numpy as np
import pandas as pd
pd.set_option('max_rows',1000)
from progressbar import ProgressBar


Rank = pd.read_excel(open('EstonianData.xlsx', 'rb'), sheet_name='Rank')






from xml.etree import ElementTree
tree = ElementTree.parse("RK2019.xml")
root = tree.getroot()


pd_DistrictName = []
pd_DistrictNumber = []
pd_DistrictQuota = []
pd_DistrictMandates = []
pd_forename = []
pd_surname = [] 
pd_candidateRegNumber = [] 
pd_candidateID = []
pd_partyName = [] 
pd_partyCode = []
pd_K_list = []
pd_P_list = []
pd_E_list = []
pd_J1P_list = []
pd_J1E_list = []
pd_J2P_list = []
pd_J2E_list = []
pd_J3P_list = [] 
pd_J3E_list = []
vote_vector = [] 
pd_adminUnit = []
pd_ehak_Code = [] 
pd_TotalVote = []



for district in list(list(list(root)[3])[0])[4]:   
    for party in list(district)[5]:
        for candidate in list(party)[2]:
            pd_DistrictName.append(list(district)[0].text)
            pd_DistrictNumber.append(int(list(district)[1].text))
            pd_DistrictQuota.append(float(list(district)[2].text))
            pd_DistrictMandates.append(list(district)[3].text)
            pd_partyName.append(list(party)[0].text)
            pd_partyCode.append(list(party)[1].text)
            pd_forename.append(list(candidate)[0].text)
            pd_surname.append(list(candidate)[1].text)
            pd_candidateRegNumber.append(int(list(candidate)[2].text))
            pd_candidateID.append(list(candidate)[3].text)
            for vote in list(candidate)[4]:
                vote_vector.append(list(vote)[1].text)
            #print(int(vote_vector[0]))
            pd_TotalVote.append(float(vote_vector[0]))
            vote_vector = []


raw_material = pd.DataFrame({'First Name':pd_forename,'Last Name':pd_surname,'RegNum':pd_candidateRegNumber,'Candidate ID':pd_candidateID,
                      'Total Votes':pd_TotalVote,'District Number':pd_DistrictNumber,'Quota':pd_DistrictQuota,'Mandates':pd_DistrictMandates,'Party Name':pd_partyName,
                      'Party Abbr.':pd_partyCode})  

raw_material['Candidates'] = raw_material['First Name'] +' '+ raw_material['Last Name']



def PartyNum(party):
    switcher={
        'EVA':1,
        'REF':2,
        'ROH':3,
        'SDE':4,
        'ERE':5,
        'EKRE' : 6,
        'EE200':7 ,
        'KESK' : 8,
        'EÜVP': 9 ,
        'IE' : 10,
        'Üksik.':11        
    }
    return switcher.get(party,"Invalid day of week")


raw_material['Party Number'] = list(map(PartyNum, raw_material['Party Abbr.']))


personalManadateElected = [263,936, 238,993,961,790,253,924,711,304,226,322,752]

districtElected = [ 1040, 701,513,313,686,342,227,1097,780,254,994,712,332,761,951,280,736,264,728,1138,545,324,288,1030,956,488,770,239,675,1156,333,1150,1020,1087,1002,228,554,265,343,986,937,269,1176,1123,241,927,1012,314,978,935,1060,476,295,
266,572,582,1166,1001,326,713,1073,781,938,942,305,945,968,791]

compensElected = [585,714,240,281,771,979,477,255,1003,1072,744,256,335,929,764,344,538,1031,1147,284]

pbar = ProgressBar()

<h1> <font color=#00539CFF> Cleaning data frame and removing unrequired details for ascertaining the result of election</font> </h1>

In [3]:
process = raw_material.copy()
clean = process[['Candidates','RegNum','Party Number','District Number','Mandates','Total Votes','Quota']]
clean

,Candidates,RegNum,Party Number,District Number,Mandates,Total Votes,Quota
0,ANDRES HERKEL,101,1,1,10,222.0,5619.10
1,ÕIE-MARI AASMÄE,102,1,1,10,43.0,5619.10
2,JÜRI PINO,106,1,1,10,36.0,5619.10
3,JAAK PROZES,103,1,1,10,35.0,5619.10
4,AGE MINKA,112,1,1,10,27.0,5619.10
...,...,...,...,...,...,...,...
1094,SIIM SUURSILD,1179,10,12,7,301.0,5887.86
1095,URMAS AAVA,1183,10,12,7,284.0,5887.86
1096,MEELIS KUKK,1180,10,12,7,279.0,5887.86
1097,JAANUS PÕLDMAA,1182,10,12,7,278.0,5887.86


In [6]:
process = raw_material.copy()
clean = process[['Candidates','RegNum','Party Number','District Number','Mandates','Total Votes','Quota']]
clean

        
def PersonalMarginVerifier(number):
    clean = process[['Candidates','RegNum','Party Number','District Number','Mandates','Total Votes','Quota']]
    for i in tqdm(range(1099)): 
        clean['Total Votes'][i:i+1] = clean['Total Votes'][i:i+1]+number
        personal = list(clean[clean['Total Votes'] >= clean['Quota']].index)
        b = set(list(clean.iloc[personal]['RegNum'].values)) == set(personalManadateElected)
        if b == False :
            print('====',i,'====',clean['RegNum'][i])
            return True
        
        else : 
            clean = process[['Candidates','RegNum','Party Number','District Number','Mandates','Total Votes','Quota']]
    return False
        
        
#for i in range(147,1000):
#    if PersonalMarginVerifier(i):
#        print("The margin size is ", i, ' for the personal mandate candidates')
#        clean
#        break
        

In [7]:
clean.iloc[1080]

Candidates         KADRI SIMSON
RegNum                     1040
Party Number                  8
District Number              12
Mandates                      7
Total Votes              5741.0
Quota                   5887.86
Name: 1080, dtype: object

<font color=#DC143C ><h1> Estonian Tally Function </h1> </font>

<ul>
    <li><font color=#148F77><h2>Personal Mandates</h2></font></li>
    <ul>
        <li>  <b>Formula </b> number of voters/number of mandates = simple quota <br> </li>
        <li>
            <b>Who shall be elected? </b> The candidate who receives more votes than the simple quota of the district or equally to it (regardless whether he or she is an independent or a party candidate) shall be elected. There are no additional requirements. 
        </li>        
    </ul>
    <li><font color=#148F77><h2>District Mandates</h2></font></li>
    <ul>
        <li>  <b>Formula </b> Number of votes for a party/district quota=number of mandates for a party in the district <br> </li>
        <li>
            <b>Who shall be elected? </b> In the district list of the party, the candidates are ranked according to the votes received. The candidate who received the greatest number of votes shall be elected, but the number of votes must be at least 10% of the simple quota.
        </li>        
    </ul>
    <li><font color=#148F77><h2>Compensation Mandates</h2></font></li>
    <ul>
        <li>
            <b>Who shall be elected? </b> The compensation mandates are distributed according to the party’s national list of candidates. The compensation mandate shall be given to the candidate who is further up the list and who has received the number of votes totalling at least 5 % of the simple quota of the district. While distributing the mandates, the candidates who have already received a personal or a district mandate shall be passed.
        </li>        
    </ul>
</ul>

In [10]:
Candidates=np.array(clean['Candidates'].values.tolist())
RegNumbers=np.array(clean['RegNum'].values.tolist())
District=np.array(clean['District Number'].values.tolist())
Votes=np.array(clean['Total Votes'].values.tolist())
Mandate=np.array(clean['Mandates'].values.tolist())
PartyCode=np.array(clean['Party Number'].values.tolist())


def truncate(number, decimals=4):
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer.")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more.")
    elif decimals == 0:
        return math.trunc(number)

    factor = 10.0 ** decimals
    return math.trunc(number * factor) / factor

ElectoralThreshold=0.05*sum(Votes)


def District_Personal_Mandate_Distributation_by_Party(district):
    district_by_party_vector=[0]*12
    for i in range(len(Candidates)):
        distQuota = clean.loc[i]['Quota']
        if (Votes[i]>= distQuota and District[i]==district):
            district_by_party_vector[PartyCode[i]]=district_by_party_vector[PartyCode[i]]+1
    return district_by_party_vector


def Elligible_Parties():
    Elligibles=[]
    for i in range(11):
        vote = clean[clean['Party Number']== i]['Total Votes'].sum()
        if vote >=ElectoralThreshold:
            Elligibles.append(i)
    return Elligibles

<h1> <font color=#00539CFF>Here is the index of candidates choosen in personal mandates and comparing it with official results </font> </h1>

In [11]:
personal = list(clean[clean['Total Votes'] >= clean['Quota']].index)
set(list(clean.iloc[personal]['RegNum'].values)) == set(personalManadateElected)

True

<h1> <font color=#00539CFF>Margins related to personal mandates </font> </h1>

In [12]:
clean['Required'] =  clean['Quota'] - clean ['Total Votes']
clean['Required'] = clean['Required'].apply(math.ceil)
clean[['RegNum','Required']].sort_values(by='Required').head(40)

<ipython-input-12-81337912d49e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean['Required'] =  clean['Quota'] - clean ['Total Votes']
<ipython-input-12-81337912d49e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean['Required'] = clean['Required'].apply(math.ceil)


,RegNum,Required
345,263,-14023
206,936,-11768
443,961,-3653
122,238,-3351
1062,790,-3282
251,253,-2348
669,993,-2336
80,924,-1683
409,711,-1341
12,226,-727


<h1> <font color=#00539CFF>  Margins for chooasing elligible parties </font> </h1>

In [13]:
ElligMargin = clean[['Party Number','Total Votes']].groupby('Party Number').sum()
ElligMargin['Required']= [ElectoralThreshold]*11 - ElligMargin['Total Votes']
ElligMargin.sort_values(by='Required')

,Total Votes,Required
Party Number,,
2,162363.0,-134305.95
8,129618.0,-101560.95
6,99671.0,-71613.95
10,64219.0,-36161.95
4,55175.0,-27117.95
7,24448.0,3609.05
3,10227.0,17830.05
5,6858.0,21199.05
1,6461.0,21596.05


<h1> <font color=#00539CFF>Here is the index of candidates choosen in district mandates and comparing it with official results </font> </h1>

In [ ]:
def District_Mandate_Party(i):
    vote = clean[clean['Party Number']== i]['Total Votes'].sum()
    if (i in Elligible_Parties()):
        print('This party has ', vote/ElectoralThreshold, 'mandates')
    else:
        print("This party is not elligible for regional mandates")
        

        
def Party_Per_District(party,district):
    sum=0
    for k in range(len(Candidates)):
        if PartyCode[k]== party and District[k]==district:
            sum = sum + Votes[k]
    return sum



def District_Seat_Counter(party, district):
    distQuota = clean[clean['District Number'] == district]['Quota'].unique()[0]
    Seat_Numer=0
    if (party in Elligible_Parties()):
        Seat_Numer=math.floor(Party_Per_District(party, district) / distQuota )
        if (Party_Per_District(party, district) % distQuota >=  0.75 * distQuota ):
            Seat_Numer+=1
    else:
        Seat_Numer=0
    return  Seat_Numer



def District_Mandate_Vector(district):
    district_mandate=[0]*12
    for party in range(1,12):
        district_mandate[party]=District_Seat_Counter(party,district)-District_Personal_Mandate_Distributation_by_Party(district)[party]
    return district_mandate



def Allocating_District_Mandate( party , district) :
    distQuota = clean[clean['District Number'] == district]['Quota'].unique()[0]
    new = clean[(clean['Party Number'] == party) & (clean['District Number'] == district)]
    final_index=list(set(new.index).difference(set(personal)))
    df = clean.loc[final_index].sort_values(by=['Total Votes']).tail(District_Mandate_Vector(district)[party])
    dff = df[df['Total Votes'] >= 0.1 * distQuota]
    return (list(dff.index))


def District_Mandates_List():
    mandates_list=[]
    for party in tqdm(Elligible_Parties()):
        for district in tqdm(range(1,13)):
            mandates_list = mandates_list + Allocating_District_Mandate(party,district)
    return(mandates_list)


#####################################
distMandList = District_Mandates_List()
#####################################

set(list(clean.iloc[distMandList]['RegNum'].values)) == set(districtElected)

 92%|█████████▏| 11/12 [00:21<00:01,  1.94s/it]

In [104]:
def District_Mandate_Party(i):
    vote = clean[clean['Party Number']== i]['Total Votes'].sum()
    if (i in Elligible_Parties()):
        print('This party has ', vote/ElectoralThreshold, 'mandates')
    else:
        print("This party is not elligible for regional mandates")
        

        
def Party_Per_District(party,district):
    sum=0
    for k in range(len(Candidates)):
        if PartyCode[k]== party and District[k]==district:
            sum = sum + Votes[k]
    return sum



def District_Seat_Counter(party, district):
    distQuota = clean[clean['District Number'] == district]['Quota'].unique()[0]
    Seat_Numer=0
    if (party in Elligible_Parties()):
        Seat_Numer=math.floor(Party_Per_District(party, district) / distQuota )
        if (Party_Per_District(party, district) % distQuota >=  0.75 * distQuota ):
            Seat_Numer+=1
    else:
        Seat_Numer=0
    return  Seat_Numer


def Party_Per_District_pd(district):
    df = clean[clean['District Number']==district][['Party Number','Total Votes']].groupby('Party Number').sum()
    df['Quota']=clean[clean['District Number']==district]['Quota']
    df['Seats']= (df['Total Votes'] / df['Quota'])
    df['0.75-Quota'] = 0.75 * df['Quota']
    df['Extra mandate'] = df['Total Votes'] % df['Quota'] >= df['0.75-Quota']
    return df.drop(list(set(range(1,11)).difference(set(Elligible_Parties()))))


def District_Mandate_Vector(district):
    district_mandate=[0]*12
    for party in range(1,12):
        district_mandate[party]=District_Seat_Counter(party,district)-District_Personal_Mandate_Distributation_by_Party(district)[party]
    return district_mandate


Party_Per_District_pd(1)

,Total Votes,Quota,Seats,0.75-Quota,Extra mandate
Party Number,,,,,
2,16299.0,5619.1,2.900642,4214.325,True
4,5683.0,5619.1,1.011372,4214.325,False
6,6361.0,5619.1,1.132032,4214.325,False
8,16950.0,5619.1,3.016497,4214.325,False
10,4950.0,5619.1,0.880924,4214.325,True
11,328.0,5619.1,0.058372,4214.325,False


<h1> <font color=#00539CFF>Here is the index of candidates choosen in compensation mandates and comparing it with official results </font> </h1>

In [ ]:

def NationalPersonalParty():
    s = np.array([0]*12)
    for district in range(13):
        s = s + np.array(District_Personal_Mandate_Distributation_by_Party(district))
    return(s)


def District_National_By_Party(party):
    s = 0
    for i in range(1,13):
        s = s + len(Allocating_District_Mandate(party, i))
    return(s)


def TotalAlloctedForParty(party):
    return (NationalPersonalParty()[party]+District_National_By_Party(party))

def RemainedMandates():
    return (101 -(len(personal)+ len (distMandList)))

################
def Compensation_Elligible_Parties():
    CompensationElligibles=[]
    for i in range(12):
        vote = clean[clean['Party Number'] == i]['Total Votes'].sum()
        if vote >=ElectoralThreshold:
            CompensationElligibles.append(i)
    return CompensationElligibles

###############
remMandate= RemainedMandates()

def ComparativeFigure(party):
    totalParty = TotalAlloctedForParty(party)
    if party in Compensation_Elligible_Parties():
        vector=[truncate(i ** 0.9,4) for i in range(totalParty +1,totalParty + remMandate+1)]
        s = np.array(vector)
        partyvote = clean[clean['Party Number'] == party]['Total Votes'].sum()
        vector=list(zip(list(map(truncate,list(partyvote /s))),[party] * remMandate))
    else :
        vector = None 
    return vector

def NationalComparativeFigures():
    nationalList=[]
    for party in tqdm(Compensation_Elligible_Parties()):
        nationalList = nationalList + ComparativeFigure(party)
    return nationalList
        
    

def RankMapping(i): 
    if list(clean[clean['RegNum']==i]['Party Number'])[0] == 11:
        return 0
    else:
        return list(Rank[Rank['RegNum'] == i]['Rank'])[0]
    
clean['Rank'] = clean['RegNum']
clean['Rank'] = list(map(RankMapping,clean['RegNum']))    
    

    
df = pd.DataFrame(NationalComparativeFigures())
removeCompensation = personal + distMandList
final_index=list(set(clean.index).difference(set(removeCompensation)))
clean_new =  clean.iloc[final_index]
dff = df.sort_values(by=0, ascending= False).head(remMandate)
CompensationSeatsPerParty = dff[1].value_counts()


    
def CompensationIndexSet():
    result=[]
    for party in Compensation_Elligible_Parties():
        compensResult = clean_new[(clean_new['Party Number'] == party) & (clean_new['Total Votes']>= 0.05* clean_new['Quota'])].sort_values(by='Rank', ascending = True).head(CompensationSeatsPerParty[party])
        result = result + list(compensResult.index)
    return result

    
clean.iloc[CompensationIndexSet()] 
set(list(clean.iloc[CompensationIndexSet()]['RegNum'].values)) == set(compensElected)